# Przykład szyfrowania i deszyfrowania z użyciem algorytmu symetrycznego


## Przygotowania

Wygenerowanie pliku testowego

In [3]:
import hashlib

def generate_text_file(file_path, text_content):
  """Generates a text file with the given content.

  Args:
    file_path: The path to the file to be created.
    text_content: The text content to be written to the file.
                   Can be multiline.
  """
  try:
    with open(file_path, 'w') as file:
      file.write(text_content)
    print(f"File '{file_path}' created successfully.")
  except Exception as e:
    print(f"An error occurred: {e}")

# code to print the contents of the file
def print_file_content(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
            print(content)
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
file_path = 'plaintext.txt'
text_content = """This is the first line.
This is the second line.
And this is the third line.
"""

def compare_files(file1_path, file2_path):
    """Compares two binary files and prints the result.

    Args:
        file1_path: Path to the first binary file.
        file2_path: Path to the second binary file.
    """
    try:
        hasher1 = hashlib.sha256()
        with open(file1_path, 'rb') as file1:
            while True:
                chunk = file1.read(4096)
                if not chunk:
                    break
                hasher1.update(chunk)

        hasher2 = hashlib.sha256()
        with open(file2_path, 'rb') as file2:
            while True:
                chunk = file2.read(4096)
                if not chunk:
                    break
                hasher2.update(chunk)

        if hasher1.hexdigest() == hasher2.hexdigest():
            print(f"Files '{file1_path}' and '{file2_path}' are identical.")
        else:
            print(f"Files '{file1_path}' and '{file2_path}' are different.")

    except FileNotFoundError:
        print(f"One or both files not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


generate_text_file(file_path, text_content)
print_file_content(file_path)


File 'plaintext.txt' created successfully.
This is the first line.
This is the second line.
And this is the third line.



## Biblioteka `pycryptodomex`

- Instalacja biblioteki (tylko w Colab).

In [2]:
%pip install pycryptodomex

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


- Definicja procedur szyfrujących i deszyfryjących plik.

Zwróć uwagę, że aby osiągnąć kompatybilność z procedurą deszyfrującą, nie tylko parametry szyfru, ale również **procedura dopełniania** oraz **procedura generacji klucza sesyjnego** muszą być identyczne. Oznacza to, że niezależnie od wykorzystywanej funkcji bibliotecznej wszystkie parametry wywołania muszą być zgodnego do najdrobniejszego szczegółu. Dlatego w kodzie źródłowym warto jawnie specyfikować wartości parametrów i nie polegać na wartościach domyślnie przyjętych przez twórców biblioteki (te mogą się zmienić w następnej wersji, co może prowadzić do błędów trudnych do wykrycia).

In [1]:
from Cryptodome.Cipher import AES
from Cryptodome.Protocol.KDF import PBKDF2
from Cryptodome.Random import get_random_bytes
from Cryptodome.Util.Padding import pad, unpad
from Cryptodome.Hash import SHA256

def encrypt_file_cryptodomex(input_file, output_file, password):
    salt = b'Kryptografia'
    key = PBKDF2(password.encode(), salt, dkLen=32, count=100000, hmac_hash_module=SHA256)
    iv = get_random_bytes(16)
    with open(input_file, 'rb') as f:
        plaintext = f.read()
    padded_plaintext = pad(plaintext, AES.block_size, style='pkcs7')
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(padded_plaintext)
    with open(output_file, 'wb') as f:
        f.write(iv + ciphertext)

def decrypt_file_cryptodomex(input_file, output_file, password):
    salt = b'Kryptografia'
    key = PBKDF2(password.encode(), salt, dkLen=32, count=100000, hmac_hash_module=SHA256)
    with open(input_file, 'rb') as f:
        data = f.read()
    iv = data[:16]
    ciphertext = data[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    padded_plaintext = cipher.decrypt(ciphertext)
    plaintext = unpad(padded_plaintext, AES.block_size, style='pkcs7')
    with open(output_file, 'wb') as f:
        f.write(plaintext)

- Zaszyfrowanie pliku testowego, a następnie jego odszyfrowanie i wyświetlenie zdeszyfrowanej zawarości.

In [4]:
encrypt_file_cryptodomex('plaintext.txt', 'ciphertext_cryptodomex.bin', 'TwojeHaslo')
decrypt_file_cryptodomex('ciphertext_cryptodomex.bin', 'decrypted.txt', 'TwojeHaslo')
compare_files('plaintext.txt', 'decrypted.txt')
print_file_content('decrypted.txt')

Files 'plaintext.txt' and 'decrypted.txt' are identical.
This is the first line.
This is the second line.
And this is the third line.



## Biblioteka `cryptography`

- Biblioteki nie trzeba instalować, również w Colab.

- Definicja procedur szyfrujących i deszyfrujących

In [ ]:
import os
from cryptography.hazmat.primitives import padding, hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

def encrypt_file_cryptography(input_file, output_file, password):
    salt = b'Kryptografia'
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())
    iv = os.urandom(16)
    with open(input_file, 'rb') as f:
        plaintext = f.read()
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_plaintext = padder.update(plaintext) + padder.finalize()
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
    with open(output_file, 'wb') as f:
        f.write(iv + ciphertext)

def decrypt_file_cryptography(input_file, output_file, password):
    salt = b'Kryptografia'
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())
    with open(input_file, 'rb') as f:
        data = f.read()
    iv = data[:16]
    ciphertext = data[16:]
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    plaintext = unpadder.update(padded_plaintext) + unpadder.finalize()
    with open(output_file, 'wb') as f:
        f.write(plaintext)

- Zaszyfrowanie pliku testowego, a następnie jego odszyfrowanie i wyświetlenie zdeszyfrowanej zawarości.

In [6]:
encrypt_file_cryptography('plaintext.txt', 'ciphertext_cryptography.bin', 'TwojeHaslo')
decrypt_file_cryptography('ciphertext_cryptography.bin', 'decrypted.txt', 'TwojeHaslo')
compare_files('plaintext.txt', 'decrypted.txt')
print_file_content('decrypted.txt')

Files 'plaintext.txt' and 'decrypted.txt' are identical.
This is the first line.
This is the second line.
And this is the third line.



## Sprawdzenie krzyżowe (cross validation)

1. Szyfrujemy `cryptodomex`, deszyfrujemy `cryptography`

In [7]:
encrypt_file_cryptodomex('plaintext.txt', 'ciphertext_cryptodomex.bin', 'TwojeHaslo')
decrypt_file_cryptography('ciphertext_cryptodomex.bin', 'decrypted.txt', 'TwojeHaslo')
compare_files('plaintext.txt', 'decrypted.txt')
print_file_content('decrypted.txt')

Files 'plaintext.txt' and 'decrypted.txt' are identical.
This is the first line.
This is the second line.
And this is the third line.



2. Szyfrujemy `cryptography`, deszyfrujemy `crytpodomex`

In [8]:
encrypt_file_cryptography('plaintext.txt', 'ciphertext_cryptography.bin', 'TwojeHaslo')
decrypt_file_cryptodomex('ciphertext_cryptography.bin', 'decrypted.txt', 'TwojeHaslo')
compare_files('plaintext.txt', 'decrypted.txt')
print_file_content('decrypted.txt')

Files 'plaintext.txt' and 'decrypted.txt' are identical.
This is the first line.
This is the second line.
And this is the third line.

